In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install pandas
!pip install gspread



In [ ]:
import pandas as pd

# Load data
greenwashing_data = pd.read_csv('/content/drive/MyDrive/ENSS_Capstone_Hillenbrand/greenwashing2.csv')

# Display the first few rows of the DataFrame
greenwashing_data.head()


,data,target
0,We strive to be environmentally friendly in ev...,1
1,Our products are made with the environment in ...,1
2,We are committed to sustainability without com...,1
3,Our goal is to reduce our carbon footprint.,1
4,We are working towards a greener future.,1


data set loaded

In [ ]:
import os
## Directories for testing and training data
#data = '/content/drive/Sharedwithme/ENSS_Capstone_Hillenbrand/greenwashing_toy'
#file_name = 'greenwashing_toy'  # Replace 'your_dataset.csv' with the actual file name
#file_path = os.path.join(data_path, file_name)
#df = pd.read_csv(file_path)

from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(greenwashing_data, test_size=0.25, random_state=100)

target = greenwashing_data['target']

# Drop the target column from the features
features = greenwashing_data.drop(columns=['target'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.5, random_state=42)


# Assuming your data is stored in a variable called 'data'
# and the target variable you want to predict is stored in a variable called 'target'

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(greenwashing_data, target, test_size=0.2, random_state=42)

# You can now use X_train and y_train for training your model
# and X_test and y_test for evaluating its performance


data split into test and train

In [ ]:
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the greenwashing_data dataset (replace this with your actual code to load the dataset)
# greenwashing_data = load_dataset()

# Assuming greenwashing_data is a dictionary containing 'texts' and 'labels'
texts = greenwashing_data['data']
labels = greenwashing_data['target']

# Preprocess the text data to mark claims containing percentage (%)
processed_texts = []
for text in texts:
    if '%' in text:
        text += ' not greenwashing'
    processed_texts.append(text)

# Tokenize the processed text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(processed_texts)
word_index = tokenizer.word_index

# Convert text data to sequences of integers
sequences = tokenizer.texts_to_sequences(processed_texts)

# Padding sequences to make them of equal length
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# Define the RNN model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(word_index) + 1, 16, input_length=max_length),
    tf.keras.layers.LSTM(16),  # LSTM layer with 16 units
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Convert lists to numpy arrays
padded_sequences = np.array(padded_sequences)
labels = np.array(labels)

# Train the model
model.fit(padded_sequences, labels, epochs=10, verbose=2)

# Evaluate the model (dummy testing data since it's a simple example)
# Replace this with your actual testing data and evaluation code
test_texts = ["Their new initiative sounds promising, but let's see if it's genuine.",
              "I'm skeptical about their claims of sustainability.",
              "This product is advertised as environmentally friendly, but is it really?"]
test_sequences = tokenizer.texts_to_sequences(test_texts)
padded_test_sequences = pad_sequences(test_sequences, maxlen=max_length, padding='post')
padded_test_sequences = np.array(padded_test_sequences)

predictions = model.predict(padded_test_sequences)
print("Predictions:", predictions)


Epoch 1/10
6/6 - 4s - loss: 0.6932 - accuracy: 0.4737 - 4s/epoch - 720ms/step
Epoch 2/10
6/6 - 0s - loss: 0.6929 - accuracy: 0.5158 - 119ms/epoch - 20ms/step
Epoch 3/10
6/6 - 0s - loss: 0.6925 - accuracy: 0.5158 - 98ms/epoch - 16ms/step
Epoch 4/10
6/6 - 0s - loss: 0.6920 - accuracy: 0.5158 - 109ms/epoch - 18ms/step
Epoch 5/10
6/6 - 0s - loss: 0.6911 - accuracy: 0.5158 - 104ms/epoch - 17ms/step
Epoch 6/10
6/6 - 0s - loss: 0.6894 - accuracy: 0.5158 - 100ms/epoch - 17ms/step
Epoch 7/10
6/6 - 0s - loss: 0.6855 - accuracy: 0.5579 - 110ms/epoch - 18ms/step
Epoch 8/10
6/6 - 0s - loss: 0.6752 - accuracy: 0.9368 - 101ms/epoch - 17ms/step
Epoch 9/10
6/6 - 0s - loss: 0.6457 - accuracy: 0.8263 - 96ms/epoch - 16ms/step
Epoch 10/10
6/6 - 0s - loss: 0.5752 - accuracy: 0.8316 - 106ms/epoch - 18ms/step
1/1 [==============================] - 0s 441ms/step
Predictions: [[0.68645155]
 [0.6718148 ]
 [0.6361394 ]]


data is now trained

In [ ]:
# Convert predictions to binary values (0 or 1)
binary_predictions = (predictions > 0.5).astype(int)

# Convert binary predictions to 1D array
binary_predictions = np.squeeze(binary_predictions)

# Dummy labels for testing
dummy_labels = np.array([1, 0, 1])  # Replace with actual labels

# Calculate accuracy
accuracy = np.mean(binary_predictions == dummy_labels)
print("Accuracy:", accuracy)


Accuracy: 0.3333333333333333


In [ ]:
# Ask the user to input text
user_input = input("Enter the text to classify: ")

# Tokenize the user input
user_sequence = tokenizer.texts_to_sequences([user_input])
padded_user_sequence = pad_sequences(user_sequence, maxlen=max_length, padding='post')

# Convert user input sequence to numpy array
padded_user_sequence = np.array(padded_user_sequence)

# Predict the label for the user input
prediction = model.predict(padded_user_sequence)


# Print the predicted label
if prediction >= 0.5:
    print("Predicted Label: Greenwashing")
else:
   print("Predicted Label: Not Greenwashing")

print(prediction)

#this does not have the level of data needed to be completely accurate in determining greenwashing content, just an example of how it would/could work


KeyboardInterrupt: Interrupted by user

put environmental claim in above to check for probabilty of greenwashing